<a href="https://colab.research.google.com/github/B10956048/113_Finance/blob/main/1119_week11_%E7%94%9F%E6%88%90%E5%BC%8FAI%E5%88%86%E6%9E%90%E8%82%A1%E7%A5%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 安裝yahoo finance套件
!pip install yfinance

In [3]:
import google.generativeai as genai  # Google生成式AI套件Gemini API
import os  # 系統檔，用來操作檔案與文件
import requests  # 用來發HTTP請求，用於網路爬蟲
from bs4 import BeautifulSoup  # 網路爬蟲套件
import numpy as np  # 數值計算
import pandas as pd  # 資料處理
import yfinance as yf  # Yahoo finance 股價資訊
import datetime as dt  # 處理日期時間
import time  # 時間延遲套件，用於控制API呼叫頻率


# 爬取股票資訊的類別
class StockInfo():
    """
    股票基本資訊爬取類別

    功能：
    1. 從台灣證券交易所爬取所有上市公司的股號、股名、產業別
    2. 提供股號查詢股名的功能

    資料來源：台灣證券交易所 (TWSE) ISIN網站
    """

    # 取得全部股票的股號、股名
    def stock_name(self):
        """
        爬取所有台股的股號、股名、產業別

        Returns:
            DataFrame: 包含「股號」、「股名」、「產業別」三欄的資料表

        技術說明：
        - 使用requests取得網頁內容
        - 使用BeautifulSoup解析HTML
        - 過濾出4碼股號的上市公司（排除ETF等其他商品）
        """
        # 向台灣證券交易所發送請求
        response = requests.get('https://isin.twse.com.tw/isin/C_public.jsp?strMode=2')
        # 使用BeautifulSoup解析HTML內容
        url_data = BeautifulSoup(response.text, 'html.parser')
        # 找出所有的表格列（tr標籤）
        stock_company = url_data.find_all('tr')

        # 資料處理：提取股號、股名、產業別
        # \u3000 是全形空白字元，用來分隔股號和股名
        # 只保留4碼的股號（過濾掉ETF、特別股等）
        data = [
            (row.find_all('td')[0].text.split('\u3000')[0].strip(),
                row.find_all('td')[0].text.split('\u3000')[1],
                row.find_all('td')[4].text.strip())
            for row in stock_company[2:] if len(row.find_all('td')[0].text.split('\u3000')[0].strip()) == 4
        ]
        # 建立DataFrame並設定欄位名稱
        df = pd.DataFrame(data, columns=['股號', '股名', '產業別'])
        return df

    # 取得股票名稱
    def get_stock_name(self, stock_id, name_df):
        """
        根據股號查詢股名

        Args:
            stock_id (str): 股票代號（例如：'2330'）
            name_df (DataFrame): 包含股號和股名的資料表

        Returns:
            str: 股票名稱（例如：'台積電'）
        """
        return name_df.set_index('股號').loc[stock_id, '股名']

In [4]:
class StockAnalysis():
    """
    股票分析類別 - 整合股價資料與Gemini AI分析

    主要功能：
    1. 取得股票價格資料（技術面）
    2. 取得股票基本面資料（營收、EPS）
    3. 使用Gemini AI進行股票分析
    4. 多檔股票比較與推薦

    設計理念：
    - 整合多元數據源（價格、基本面）
    - 利用LLM進行智能分析
    """

    def __init__(self, gemini_api_key):
        """
        初始化股票分析系統

        Args:
            gemini_api_key (str): Google Gemini API金鑰

        初始化步驟：
        1. 設定Gemini API
        2. 建立生成模型實例
        """
        # 設定Gemini API金鑰
        genai.configure(api_key=gemini_api_key)
        """
        1. Gemini 2.5 Pro - 進階思考模型
        2. Gemini 2.5 Flash - 快速且智慧
        3. Gemini 2.5 Flash-Lite - 速度最快的 Flash 模型，可提高成本效益和輸送量。
        """
        # 使用Gemini 2.5 Pro模型（高階推理能力）
        self.model = genai.GenerativeModel('gemini-2.5-pro')
        # 建立股票資訊物件
        self.stock_info = StockInfo()
        # 預先載入所有股票名稱（避免重複爬取）
        self.name_df = self.stock_info.stock_name()


    def stock_price(self, stock_id, days=15):
        """
        取得股票近期價格資料（技術面分析基礎）

        Args:
            stock_id (str): 股票代號（例如：'2330'）
            days (int): 要取得的天數（預設15天）

        Returns:
            dict: 包含日期、收盤價、每日報酬率的字典

        技術說明：
        - 使用yfinance API取得Yahoo Finance資料
        - 計算每日報酬率 = (今日收盤-昨日收盤)/昨日收盤
        - 自動加上.TW後綴（台股代碼格式）
        """

        # 台股代碼需要加上.TW後綴
        stock_id += '.TW'
        # 計算日期區間
        end = dt.date.today() # 今天
        start = end - dt.timedelta(days=days) # 往前推N天

        # 禁用 FutureWarning
        import warnings
        warnings.filterwarnings('ignore', category=FutureWarning)

        # 下載股價資料
        # auto_adjust=True 會自動調整除權息的價格
        df = yf.download(stock_id, start=start, auto_adjust=True)

        # 將欄位名稱改為中文（方便閱讀）
        df.columns = ['開盤價', '最高價', '最低價', '收盤價', '成交量']
        # 建立回傳的資料結構
        data = {
            '日期': df.index.strftime('%Y-%m-%d').tolist(), # 格式化日期
            '收盤價': df['收盤價'].tolist(), # 收盤價列表
            '每日報酬': df['收盤價'].pct_change().tolist() # 報酬率 = 價格變化百分比
        }
        return data


    def stock_fundamental(self, stock_id):
        """
        取得股票基本面資料（價值投資分析基礎）

        Args:
            stock_id (str): 股票代號

        Returns:
            dict: 包含季度日期、營收成長率、EPS、EPS季增率

        基本面指標說明：
        - 營收成長率：季度營收相比上一季的成長百分比
        - EPS (每股盈餘)：公司獲利能力的核心指標
        - EPS季增率：EPS相比上一季的成長百分比

        注意事項：
        - 某些股票可能無基本面資料（新上市或資料不全）
        - 使用try-except確保程式不會因單一股票錯誤而中斷
        """

        stock_id += '.TW'
        stock = yf.Ticker(stock_id)

        try:
            # 計算季度營收成長率
            # pct_change(-1)：與前一期（-1）比較的百分比變化
            # fill_method=None：不填補缺失值
            quarterly_revenue_growth = np.round(
                stock.quarterly_financials.loc['Total Revenue'].pct_change(-1, fill_method=None).dropna().tolist(),
                2
            )
            # 取得季度EPS
            quarterly_eps = np.round(
                stock.quarterly_financials.loc['Basic EPS'].dropna().tolist(),
                2
            )
            # 計算EPS季增率
            quarterly_eps_growth = np.round(
                stock.quarterly_financials.loc['Basic EPS'].pct_change(-1, fill_method=None).dropna().tolist(),
                2
            )
            # 取得日期並格式化
            dates = [date.strftime('%Y-%m-%d') for date in stock.quarterly_financials.columns]
            # 組合資料（確保長度一致）
            data = {
                '季日期': dates[:len(quarterly_revenue_growth)],
                '營收成長率': quarterly_revenue_growth.tolist(),
                'EPS': quarterly_eps.tolist(),
                'EPS 季增率': quarterly_eps_growth.tolist()
            }
        except Exception as e:
            # 如果取得資料失敗，回傳空資料
            print(f"無法取得 {stock_id} 基本面資料: {e}")
            data = {'季日期': [], '營收成長率': [], 'EPS': [], 'EPS 季增率': []}

        return data

    def _get_reply(self, content_msg):
        """
        呼叫Gemini API並處理各種回應狀況（內部方法）

        Args:
            content_msg (str): 要傳送給Gemini的提示詞（prompt）

        Returns:
            str: Gemini的回應文字或錯誤訊息

        錯誤處理機制：
        1. finish_reason = 1 (STOP)：正常完成
        2. finish_reason = 2 (MAX_TOKENS)：超過長度限制
        3. finish_reason = 3 (SAFETY)：觸發安全政策
        4. finish_reason = 4 (RECITATION)：引用問題
        5. HTTP 429：API速率限制（免費版：每分鐘5次，每天25次）
        6. HTTP 503：服務暫時不可用
        """

        try:
            # 呼叫Gemini API生成內容
            response = self.model.generate_content(
                content_msg,
                generation_config=genai.types.GenerationConfig(
                    max_output_tokens=10000,  # 最大輸出token數（避免過長）
                    temperature=1.0,  # 溫度參數（1.0 = 較有創意，0.0 = 較保守）
                )
            )

            # 檢查是否有回應候選
            if not response.candidates:
                return "模型未返回任何回應"

            candidate = response.candidates[0]
            finish_reason = candidate.finish_reason

            # finish_reason是整數值，代表不同的完成狀態：
            # 0 = UNSPECIFIED (未指定)
            # 1 = STOP (正常完成)
            # 2 = MAX_TOKENS (達到token限制)
            # 3 = SAFETY (安全過濾)
            # 4 = RECITATION (引用問題)
            # 5 = OTHER (其他原因)

            # 根據不同的完成原因處理
            if finish_reason == 1:  # STOP - 正常完成
                # 正常完成
                return candidate.content.parts[0].text

            elif finish_reason == 2:  # MAX_TOKENS - 達到長度限制
                try:
                    partial_text = candidate.content.parts[0].text if candidate.content.parts else ""
                    return f"{partial_text}\n\n [回應被截斷，已達到長度限制]"
                except:
                    return "回應超過長度限制。建議：1)增加 max_output_tokens 2)簡化 prompt"

            elif finish_reason == 3:  # SAFETY - 安全政策
                return "模型因安全政策無法提供回應"

            elif finish_reason == 4:  # RECITATION - 引用問題
                return "回應因引用問題被過濾"

            else:
                return f"未知完成原因: {finish_reason}"

        except Exception as e:
            # 處理API錯誤
            error_msg = str(e)
            # 速率限制錯誤（最常見）
            if "429" in error_msg or "RESOURCE_EXHAUSTED" in error_msg:
                return "達到 API 速率限制（免費層：5次/分鐘，25次/天）"
            # 服務暫時不可用
            elif "503" in error_msg:
                return "服務暫時不可用，請稍後重試"
            # 其他錯誤
            else:
                return f"API 錯誤: {error_msg}"


    def stock_gimini_analysis(self, stock_id):
        """
        使用Gemini AI分析單支股票（核心功能）

        Args:
            stock_id (str): 股票代號

        Returns:
            str: Gemini生成的股票分析報告

        分析流程：
        1. 取得股票名稱
        2. 收集技術面資料（價格、報酬率）
        3. 收集基本面資料（營收、EPS）
        4. 將資料摘要後傳送給Gemini
        5. 讓Gemini扮演金融分析師角色，產生專業分析

        Prompt設計要點：
        - 角色設定：明確告訴模型扮演「金融分析師」
        - 數據提供：精簡但完整的關鍵資訊
        - 輸出要求：技術面+基本面+風險評估
        - 語言指定：繁體中文
        """
        # 取得股票名稱
        stock_name = self.stock_info.get_stock_name(stock_id, self.name_df)

        # 收集股價資料
        price_data = self.stock_price(stock_id)
        print('資料收集完畢')
        # 收集基本面資料
        stock_value_data = self.stock_fundamental(stock_id)

        # --- 資料摘要製作 ---
        # 目的：將大量資料濃縮成關鍵指標，避免超過token限制
        summary_parts = []

        # 股價摘要：最新價格 + 兩週漲跌幅
        if price_data and price_data['收盤價']:
            latest = price_data['收盤價'][-1] # 最新收盤價
            first = price_data['收盤價'][0] # 首日收盤價
            change = ((latest - first) / first) * 100 # 計算漲跌百分比
            summary_parts.append(f"股價：{latest:.2f} (兩週 {change:+.2f}%)")

        # 基本面摘要：最新一季的營收成長和EPS
        if stock_value_data and stock_value_data['季日期']:
            q = stock_value_data['季日期'][0] # 季度
            rev = stock_value_data['營收成長率'][0] # 營收成長率
            eps = stock_value_data['EPS'][0] # EPS
            summary_parts.append(f"基本面({q})：營收成長{rev*100:.1f}%, EPS {eps:.2f}")

        # 組合所有摘要
        combined = "；".join(summary_parts)

        # ============ 作業修改區 START ============
        # --- 建構Prompt ---
        # 這是Prompt Engineering的關鍵：如何引導LLM產生專業分析
        """
        'conservative': '''你是保守型財務顧問...
        1. 優先評估下檔風險
        2. 關注股息穩定性
        3. 避免高波動股票...''',

        'aggressive': '''你是成長型投資顧問...
        1. 關注營收成長動能
        2. 尋找突破訊號...''',

        'balanced': '''你是專業金融分析師...
        平衡評估機會與風險...'''
        """
        content_msg = f'''假設你是金融分析師，根據以下股票數據生成投資建議{stock_name}。
資料：{combined}

    請提供詳細的技術面和基本面見解，並考慮風險因素。繁體中文回應。'''
        # ============ 作業修改區 END ============
        # 呼叫Gemini API
        reply = self._get_reply(content_msg)
        return reply


    def stock_gimini_sort(self, message):
        """
        讓Gemini為多檔股票評分並排序

        Args:
            message (dict): 包含多檔股票分析報告的字典

        Returns:
            str: Gemini生成的評分排序結果

        評分標準：
        - 50分為基準
        - 正面因素加分（例如：營收成長、技術面轉強）
        - 負面因素扣分（例如：虧損、下跌趨勢）
        - 綜合技術面+基本面+市場環境

        應用場景：投資組合篩選、資金配置決策
        """

        content_msg = f'''你是專業股票分析師，根據趨勢報告評分(0-100)。
50分為基準，正面消息加分，負面消息扣分。
排序所有股票。{str(message)}(繁體中文)'''
        reply = self._get_reply(content_msg)
        return reply


    def stock_gimini_choice(self, message):
        """
        讓Gemini從多檔股票中選出最佳投資標的

        Args:
            message (dict): 包含多檔股票分析報告的字典

        Returns:
            str: Gemini選出的最佳股票及理由

        決策邏輯：
        - 即使所有選項都不理想，也要選出相對最佳的
        - 需要說明選擇理由（風險、報酬、時機等）
        - 模擬真實的投資決策情境

        教學價值：
        - 理解LLM如何進行多維度評估
        - 學習投資決策的思考框架
        """

        content_msg = f'''你是專業證券分析師，從趨勢報告中選出最適合投資的一檔股票。
即使都不理想也要選一檔，說明理由。{str(message)}(繁體中文)'''
        reply = self._get_reply(content_msg)
        return reply

In [12]:
# 【重要】請修改為你自己的Gemini API Key
# ============ 作業修改區 START ==========
gemini_api_key = '輸入API'
# ============ 作業修改區 END ============
# 建立股票分析物件
stock_analysis = StockAnalysis(gemini_api_key)

# 單支股價分析

In [ ]:
# ============ 作業修改區 START ==========
# 改成想分析的股票代號
reply = stock_analysis.stock_gimini_analysis('2230')
# ============ 作業修改區 END ============
print(reply)

[*********************100%***********************]  1 of 1 completed


資料收集完畢
無法取得 0050.TW 基本面資料: 'Total Revenue'


# 收集多檔股票的趨勢報告

In [ ]:
# 建立股票清單
# ============ 作業修改區 START ============
# 請修改為你想分析的股票清單（建議3-5檔，避免超過API限制）
# 台股股票參考：
# 2330 台積電, 2317 鴻海, 2454 聯發科, 2412 中華電, 2881 富邦金
# 2382 廣達, 2303 聯電, 3711 日月光投控, 2891 中信金, 2886 兆豐金
stock_list = ['2330', '2535', '3041']
# ============ 作業修改區 END ============

# 設定儲存路徑
today_time = dt.date.today().strftime('%Y%m%d')
path = './StockGemini/TrendReport/'
os.makedirs(path, exist_ok=True)

# 建立多檔股票的趨勢報告並儲存
content = {}

print(f"\n{'='*70}")
print(f"開始分析 {len(stock_list)} 檔股票")
print(f"{'='*70}\n")

# 批次處理每一檔股票
for i, stock in enumerate(stock_list, 1):
    print(f"[{i}/{len(stock_list)}] 處理股票: {stock}")

    file_path = f"{path}trend_{stock}_{today_time}.txt"

    if not os.path.exists(file_path):
        try:
            # 分析股票
            print(f"   正在分析...")
            analysis = stock_analysis.stock_gimini_analysis(stock_id=stock)

            # 儲存結果
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(analysis)

            content[stock] = analysis
            print(f"   分析完成並已儲存\n")

            # 速率限制控制
            if i < len(stock_list):
                wait_time = 12
                print(f"   等待 {wait_time} 秒以符合 API 速率限制...\n")
                time.sleep(wait_time)

        except Exception as e:
            error_msg = f"錯誤: {str(e)}"
            print(f"   {error_msg}\n")
            content[stock] = error_msg

    else:
        # 讀取已存在的分析
        with open(file_path, "r", encoding="utf-8") as f:
            content[stock] = f.read()
        print(f"   從快取讀取\n")

print(f"{'='*70}")
print("所有股票分析完成！")
print(f"{'='*70}\n")

print("\n" + "="*70)
print("分析結果")
print("="*70 + "\n")

for stock_id, analysis in content.items():
    stock_name = stock_analysis.stock_info.get_stock_name(stock_id, stock_analysis.name_df)

    print("="*70)
    print(f"【{stock_id} - {stock_name}】")
    print("="*70)
    print(analysis)
    print("\n")


開始分析 3 檔股票

[1/3] 處理股票: 2330
   正在分析...


[*********************100%***********************]  1 of 1 completed


資料收集完畢
   分析完成並已儲存

   等待 12 秒以符合 API 速率限制...

[2/3] 處理股票: 2535
   正在分析...


[*********************100%***********************]  1 of 1 completed


資料收集完畢
   分析完成並已儲存

   等待 12 秒以符合 API 速率限制...

[3/3] 處理股票: 3041
   正在分析...


[*********************100%***********************]  1 of 1 completed


資料收集完畢
   分析完成並已儲存

所有股票分析完成！


分析結果

【2330 - 台積電】
好的，收到您的請求。作為一名金融分析師，我將根據您提供的數據，為您生成一份關於台積電（TSMC）的詳細投資分析報告。

---

### **台積電 (TSMC) 投資分析報告**

**報告日期：** 2024年6月10日
**分析師：** [您的AI金融分析師]

#### **摘要與投資建議**

綜合技術面、基本面與估值分析，我們對台積電的投資建議如下：

*   **長期投資者 (持有週期 > 1年)：** **謹慎增持 (Cautious Accumulate)**。
    台積電在全球半導體產業的領導地位穩固，長期成長趨勢（AI、HPC）明確。近期股價回檔 -7.89%，對於尋求長期佈局的投資者而言，可視為一個**分批建立部位**的機會。然而，目前的估值偏高，需有承受短期波動的心理準備。

*   **短期交易者 (持有週期 < 3個月)：** **中性觀望 (Neutral/Wait-and-See)**。
    技術面呈現短期弱勢，賣壓尚未完全消化。在股價出現明確的止跌回穩訊號（例如：帶量長紅K線、站穩關鍵支撐位）之前，建議保持觀望，避免追高殺低。

#### **數據概覽**

*   **當前股價：** 1400.00 元
*   **近期表現：** 兩週內下跌 -7.89%
*   **基本面預測 (至 2025-09-30)：**
    *   營收成長率：6.0%
    *   每股盈餘 (EPS)：15.36 元

---

### **一、 技術面分析 (Technical Analysis)**

**觀點：短期趨勢偏弱，尋求支撐**

1.  **價格動能與趨勢：** 股價在兩週內下跌近8%，顯示市場出現了顯著的獲利了結或避險賣壓。這打破了先前的強勁上漲動能，使短期趨勢由多轉空或進入盤整階段。目前股價處於一個**修正回檔**的格局。

2.  **關鍵支撐位：** 1400.00元本身可能是一個重要的心理關卡。投資者需密切關注此價位是否能守住。若跌破，下一個支撐區間可能落在前波的整理平台或重要的移動平均線（例如：季線、半年線）。

3.  **指標觀察：**
    *   **相對強弱指數 (RSI)：** 快速下跌可能

# 推薦一檔股票

In [ ]:
reply = stock_analysis.stock_gimini_choice(str(content))
print(reply)

好的，身為專業證券分析師，我將仔細評估您提供的三份趨勢報告，並在「即使都不理想也要選一檔」的前提下，為您選出最適合投資的標的，並闡述完整的分析邏輯。

---

### **總結：最終投資選擇**

我選擇 **3041 揚智 (ALi Corporation)** 作為這三檔中最具投資潛力的股票。

這是一個基於「**高風險、高回報潛力**」的策略性選擇。雖然揚智目前仍在虧損且短期股價疲弱，但其驚人的**預期營收成長率 (+53.0%)** 提供了一個其他兩檔股票所缺乏的關鍵要素：**一個足以扭轉局面的強大成長故事（Turnaround Story）**。

### **綜合比較分析**

為了做出決策，我將三家公司並列比較其核心優劣勢：

| 項目 | 2330 台積電 | 2535 達欣工 | **3041 揚智 (選擇)** |
| :--- | :--- | :--- | :--- |
| **投資主軸** | 穩定龍頭、AI趨勢 | 傳統營建、景氣循環 | **轉機故事、高成長潛力** |
| **股價動能** | 短期回檔 (-7.89%) | 短期回檔 (-6.70%) | **短期重挫 (-12.89%)** |
| **營收成長預期** | 穩健 (+6.0%) | **衰退 (-13.0%)** | **爆發性 (+53.0%)** |
| **獲利能力預期**| 強勁 (EPS 15.36) | 尚可 (EPS 1.76) | **虧損 (EPS -1.02)** |
| **估值 (預估P/E)**| **極度昂貴 (約 91 倍)** | **顯著偏高 (約 35 倍)** | N/A (虧損中) |
| **最大優勢** | 產業護城河深厚 | 仍有獲利能力 | **無可比擬的成長預期** |
| **最大風險** | **估值過高，幾乎沒有容錯空間** | **營收衰退，前景不明** | **無法轉虧為盈的執行風險** |
| **分析師評級** | **規避 (因估值)** | **賣出/規避** | **投機性買入 (Speculative Buy)** |

---

### **詳細決策流程與理由**

我的決策過程是透過「**刪去法**」來排除最不具吸引力的選項，最終選出風險報酬比相對最佳的標的。

#### **第一步：首

# 推薦股票的評分排序

In [ ]:
reply = stock_analysis.stock_gimini_sort(str(content))
print(reply)

好的，收到您的指示。我將以專業股票分析師的角色，根據您提供的三份詳細報告，進行趨勢評分並排序。

我的評分模型如下：
*   **基準分：** 50分 (中性)
*   **評分維度：** 結合**基本面前景**、**估值合理性**、**短期技術面**與**風險係數**四個層面進行加減分。
*   **核心邏輯：** 尋找基本面強勁、估值合理、風險可控的標的。高成長預期可加分，但若伴隨極高估值或虧損，則會進行風險折抵。

---

### **股票趨勢評分與排序總覽**

| 排名 | 股票代號 | 公司名稱 | 趨勢分數 (0-100) | 核心觀點 |
| :--- | :--- | :--- | :--- | :--- |
| **1** | **2330** | **台積電** | **56 分** | **優質但昂貴**：長期基本面無可挑剔，但極端估值與短期技術弱勢限制了分數。 |
| **2** | **3041** | **揚智** | **37 分** | **高風險轉機**：營收高成長是唯一亮點，但持續虧損與技術面破底是重大警訊。 |
| **3** | **2535** | **達欣工** | **5 分** | **衰退且昂貴**：營收預期衰退，估值卻處於高位，風險回報比極差。 |

---

### **個股評分解析**

#### **1. 台積電 (2330) - 趨勢分數：56 分**

台積電是一家卓越的公司，但目前的價格使其成為一項充滿挑戰的投資。分數略高於中性，反映了其強大的基本面與高昂估值之間的拉鋸戰。

*   **正面因素 (加分):**
    *   **產業領導地位 (+20分):** 報告明確指出其「領導地位穩固」、「護城河極深」，這是最核心的價值所在。
    *   **搭上AI長期趨勢 (+10分):** AI與HPC是未來數年最確定的成長引擎，台積電是主要受益者。
    *   **獲利能力強勁 (+5分):** 預估EPS 15.36元，顯示公司擁有強大的盈利基礎。
    *   **營收穩健成長 (+3分):** 在龐大基數上仍有6%的成長預期，表現穩健。

*   **負面因素 (扣分):**
    *   **估值極度昂貴 (-20分):** 報告計算出**91倍**的預期本益比，並強調其「極端